In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Download the Data

In [18]:
# cpi = pd.read_csv(r'data/cpi.csv')
# fed_funds_rate = pd.read_csv(r'data/fed_funds_rate.csv')
# gdp = pd.read_csv(r'data/gdp.csv')
# sentiment = pd.read_csv(r'data/sentiment.csv')
# SP_price_action = pd.read_csv(r'data/SP_price_action.csv')
# u_rate = pd.read_csv(r'data/u_rate.csv')

# Clean the data
Get the data ready for a format to combine the data. Get the data and maybe some lag information.

In [28]:
# CPI 
cpi = pd.read_csv(r'data/cpi.csv')
cpi.columns = ['date', 'cpi_base']
cpi['date'] = pd.to_datetime(cpi['date'], format='%Y-%m-%d')
cpi['1yr_change'] = cpi['cpi_base'].diff()
cpi['1yr_cpi_pct_change'] = cpi['cpi_base'].pct_change() * 100
cpi.head()

,date,cpi_base,1yr_change,1yr_cpi_pct_change
0,1960-01-01,1.457976,NaN,NaN
1,1961-01-01,1.070724,-0.387252,-26.560920
2,1962-01-01,1.198773,0.128049,11.959121
3,1963-01-01,1.239669,0.040896,3.411493
4,1964-01-01,1.278912,0.039242,3.165533


In [36]:
fed_funds_rate = pd.read_csv(r'data/fed_funds_rate.csv')
fed_funds_rate.columns = ["date", "base_fed_funds_rate"]
fed_funds_rate['date'] = pd.to_datetime(fed_funds_rate['date'], format='%Y-%m-%d')
fed_funds_rate['1day_fed_funds_change'] = fed_funds_rate['base_fed_funds_rate'].diff()
fed_funds_rate['1yr_fed_funds_diff'] = fed_funds_rate['base_fed_funds_rate'] - fed_funds_rate['base_fed_funds_rate'].shift(365) 
fed_funds_rate['1yr_fed_funds_pct_change'] = (fed_funds_rate['base_fed_funds_rate'] - fed_funds_rate['base_fed_funds_rate'].shift(365))/fed_funds_rate['base_fed_funds_rate'].shift(365)
fed_funds_rate['1m_fed_funds_diff'] = fed_funds_rate['base_fed_funds_rate'] - fed_funds_rate['base_fed_funds_rate'].shift(30) 
fed_funds_rate

,date,base_fed_funds_rate,1day_fed_funds_change,1yr_fed_funds_diff,1yr_fed_funds_pct_change,1m_fed_funds_diff
0,2019-06-06,2.37,NaN,NaN,NaN,NaN
1,2019-06-07,2.37,0.0,NaN,NaN,NaN
2,2019-06-08,2.37,0.0,NaN,NaN,NaN
3,2019-06-09,2.37,0.0,NaN,NaN,NaN
4,2019-06-10,2.37,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...
1823,2024-06-02,5.33,0.0,0.25,0.049213,0.0
1824,2024-06-03,5.33,0.0,0.25,0.049213,0.0
1825,2024-06-04,5.33,0.0,0.25,0.049213,0.0
1826,2024-06-05,5.33,0.0,0.25,0.049213,0.0


In [21]:
gdp = pd.read_csv(r'data/gdp.csv')
gdp.columns = ["date", 'gdp_base']
gdp['date'] = pd.to_datetime(gdp['date'], format='%Y-%m-%d')
gdp = gdp[gdp['date'].dt.month == 1] # Came in by quarter, easiest thing to do was just take the first day of the year 
gdp['1yr_gdp_pct_change'] = gdp['gdp_base'].pct_change() * 100
gdp

,date,gdp_base,1yr_gdp_pct_change
0,1947-01-01,2182.681,NaN
4,1948-01-01,2239.682,2.611513
8,1949-01-01,2260.807,0.943214
12,1950-01-01,2346.104,3.772856
16,1951-01-01,2593.967,10.564877
...,...,...,...
292,2020-01-01,20665.553,1.226555
296,2021-01-01,20990.541,1.572607
300,2022-01-01,21738.871,3.565082
304,2023-01-01,22112.329,1.717927


In [22]:
sentiment = pd.read_csv(r'data/sentiment.csv')
sentiment.columns = ['date', 'base_sentiment']
sentiment = sentiment[sentiment['base_sentiment'] != '.']
sentiment['base_sentiment'] = sentiment['base_sentiment'].astype(float)

# There were a few months that were spread out and not one of them was the first of the year. I have taken the yearly average and placed that as the first of the year
sentiment['date'] = pd.to_datetime(sentiment['date'], format='%Y-%m-%d')
sentiment['year'] = sentiment['date'].dt.year
yearly_avg_sentiment = sentiment.groupby('year')['base_sentiment'].mean().reset_index()
yearly_avg_sentiment['date'] = pd.to_datetime(yearly_avg_sentiment['year'].astype(str) + '-01-01')
sentiment = yearly_avg_sentiment[['date', 'base_sentiment']]


sentiment['1yr_sentiment_pct_change'] = sentiment['base_sentiment'].pct_change() * 100

sentiment.head()

,date,base_sentiment,1yr_sentiment_pct_change
0,1952-01-01,86.200000,NaN
1,1953-01-01,84.066667,-2.474865
2,1954-01-01,83.966667,-0.118953
3,1955-01-01,98.233333,16.990869
4,1956-01-01,99.433333,1.221581


In [23]:
u_rate = pd.read_csv(r'data/u_rate.csv')
u_rate.columns = ['date', 'base_u_rate']
u_rate['date'] = pd.to_datetime(u_rate['date'], format='%Y-%m-%d')
u_rate['1m_u_rate_pct_change'] = u_rate['base_u_rate'].pct_change() * 100
u_rate['1yr_u_rate_pct_diff'] = u_rate['base_u_rate'].pct_change(periods=12) * 100

u_rate.head()

,date,base_u_rate,1m_u_rate_pct_change,1yr_u_rate_pct_diff
0,1948-01-01,3.4,NaN,NaN
1,1948-02-01,3.8,11.764706,NaN
2,1948-03-01,4.0,5.263158,NaN
3,1948-04-01,3.9,-2.500000,NaN
4,1948-05-01,3.5,-10.256410,NaN


# S&P 500 processing 
As this is the heart of our data and it is daily. For further analysis also I would want the data processed slightly differently. 

In [24]:
sp_price_action = pd.read_csv(r'data/SP_price_action.csv')
sp_price_action = sp_price_action[['Date', 'Adj Close', 'Volume']]
sp_price_action.columns = ['date', 'price', 'volume']
sp_price_action['date'] = pd.to_datetime(sp_price_action['date'])
sp_price_action = sp_price_action[~sp_price_action['price'].isna()]
sp_price_action = sp_price_action[sp_price_action['volume'] > 0]
sp_price_action.head()

,date,price,volume
5496,1950-01-03,16.66,1260000.0
5497,1950-01-04,16.85,1890000.0
5498,1950-01-05,16.93,2550000.0
5499,1950-01-06,16.98,2010000.0
5500,1950-01-09,17.08,2520000.0


## Adding columns to the data

In [25]:
#Columns to add
# Set 'date' as index
sp_price_action.set_index('date', inplace=True)

# Calculate 1-Day Price Difference
sp_price_action['price_1d_diff'] = sp_price_action['price'].diff()

# Calculate 1-Month Price Difference
sp_price_action['price_1m_diff'] = sp_price_action['price'] - sp_price_action['price'].shift(21)  # Approx. 21 trading days in a month

# Calculate 1-Day Volume Difference
sp_price_action['volume_1d_diff'] = sp_price_action['volume'].diff()

# Calculate 1-Month Volume Difference
sp_price_action['volume_1m_diff'] = sp_price_action['volume'] - sp_price_action['volume'].shift(21)  # Approx. 21 trading days in a month

# Calculate Percentage Change in Price
sp_price_action['price_pct_change'] = sp_price_action['price'].pct_change() * 100

# Calculate Rolling Mean Price (7-day)
sp_price_action['rolling_mean_price_7d'] = sp_price_action['price'].rolling(window=7).mean()

# Calculate Rolling Standard Deviation Price (7-day)
sp_price_action['rolling_std_price_7d'] = sp_price_action['price'].rolling(window=7).std()

# Calculate Rolling Mean Volume (7-day)
sp_price_action['rolling_mean_volume_7d'] = sp_price_action['volume'].rolling(window=7).mean()

# Calculate Rolling Standard Deviation Volume (7-day)
sp_price_action['rolling_std_volume_7d'] = sp_price_action['volume'].rolling(window=7).std()

# Calculate Price Momentum (14-day)
sp_price_action['momentum_14d'] = sp_price_action['price'] - sp_price_action['price'].shift(14)

# Calculate Exponential Moving Average (20-day)
sp_price_action['ema_20d'] = sp_price_action['price'].ewm(span=20, adjust=False).mean()

# Calculate Year-over-Year Price Change
sp_price_action['price_yoy_diff'] = sp_price_action['price'] - sp_price_action['price'].shift(365)

# Calculate Year-over-Year Volume Change
sp_price_action['volume_yoy_diff'] = sp_price_action['volume'] - sp_price_action['volume'].shift(365)

# Calculate Year-over-Year Percentage Price Change
sp_price_action['price_yoy_pct_change'] = sp_price_action['price'].pct_change(periods=365) * 100

# Calculate Year-over-Year Percentage Volume Change
sp_price_action['volume_yoy_pct_change'] = sp_price_action['volume'].pct_change(periods=365) * 100

sp_price_action = sp_price_action.reset_index()

# Building Modeling Datasets

# 1 Year Dataset
With so many varying time frames, the one that appears the most common is the 1 year timeframe. I will build a dataset to model that one off of. 

In [41]:
# Get all of the year related columns in the dataset
year_sp_price_action = sp_price_action[['date', 'price_yoy_diff', 'volume_yoy_diff', 'price_yoy_pct_change', 'volume_yoy_pct_change']]
year_u_rate = u_rate[['date', '1yr_u_rate_pct_diff']]
year_sentiment = sentiment[['date', '1yr_sentiment_pct_change']]
year_gdp = gdp[['date', '1yr_gdp_pct_change']]
year_fed_funds_rate = fed_funds_rate[['date', '1yr_fed_funds_pct_change']]
year_cpi = cpi[['date', '1yr_cpi_pct_change']]

,date,1yr_cpi_pct_change
0,1960-01-01,NaN
1,1961-01-01,-26.560920
2,1962-01-01,11.959121
3,1963-01-01,3.411493
4,1964-01-01,3.165533
...,...,...
58,2018-01-01,14.669350
59,2019-01-01,-25.807645
60,2020-01-01,-31.929283
61,2021-01-01,280.829952


In [48]:
# Group them all together into one dataset
year_df1 = pd.merge(year_sp_price_action, year_u_rate, on='date', how='outer')
year_df2 = pd.merge(year_df1, year_sentiment, on='date', how='outer')
year_df3 = pd.merge(year_df2, year_gdp, on='date', how='outer')
year_df4 = pd.merge(year_df3, year_fed_funds_rate, on='date', how='outer')
year_df5 = pd.merge(year_df4, year_cpi, on='date', how='outer')
year_df5

,date,price_yoy_diff,volume_yoy_diff,price_yoy_pct_change,volume_yoy_pct_change,1yr_u_rate_pct_diff,1yr_sentiment_pct_change,1yr_gdp_pct_change,1yr_fed_funds_pct_change,1yr_cpi_pct_change
0,1950-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1950-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1950-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1950-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
19621,2024-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049213,NaN
19622,2024-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049213,NaN
19623,2024-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049213,NaN
19624,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049213,NaN


The year data requires some extra work because the data is released at different times. I started by taking just the first of the year but that ended up not working because the stock market is not open on Jan 1, when lots of the year data is postmarked. So in order to also further clean the data, I took an average of the entire year, ignoting all the NaN. This way I have a dataset to work on

In [64]:
year_df5['year'] = year_df5['date'].dt.year
final_year_averages = year_df5.groupby('year').mean(numeric_only=True)
final_year_averages = final_year_averages[~final_year_averages['price_yoy_diff'].isna()]
final_year_averages

,price_yoy_diff,volume_yoy_diff,price_yoy_pct_change,volume_yoy_pct_change,1yr_u_rate_pct_diff,1yr_sentiment_pct_change,1yr_gdp_pct_change,1yr_fed_funds_pct_change,1yr_cpi_pct_change
year,,,,,,,,,
1951,5.123985,-3.953383e+05,29.029773,-15.020485,-35.340175,NaN,10.564877,NaN,NaN
1952,3.988280,-6.749200e+05,19.887514,-24.103316,-7.507572,NaN,5.165332,NaN,NaN
1953,1.140359,1.243028e+04,4.972609,5.171929,-2.239049,-2.474865,6.189877,NaN,NaN
1954,4.608373,8.485317e+05,18.549472,76.504025,97.397452,-0.118953,-1.781925,NaN,NaN
1955,14.468690,9.003571e+05,55.606807,74.416946,-21.375729,16.990869,6.170199,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020,416.037672,1.306877e+09,14.889219,38.676798,121.924519,-15.046015,1.226555,-0.953299,-31.929283
2021,1254.378536,-1.543642e+08,42.304832,8.150914,-17.346642,-4.813490,1.572607,-0.196831,280.829952
2022,304.605501,3.890279e+07,9.473566,5.095595,-30.502669,-24.017608,3.565082,20.089552,70.349941


## Adding the dependent variable
buy - if the next years price_yoy_pct_change is over 10  
hold - if the next years price_yoy_pct_change is 5-10  
sell - if the next years price_yoy_pct_change is under 5  

In [68]:
# Calculate the price_yoy_pct_change for the next year
final_year_averages['next_year_price_yoy_pct_change'] = final_year_averages['price_yoy_pct_change'].shift(-1)

def determine_action(change):
    if pd.isna(change):
        return None
    elif change > 10:
        return 'buy'
    elif 5 <= change <= 10:
        return 'hold'
    else:
        return 'sell'

final_year_averages['action'] = final_year_averages['next_year_price_yoy_pct_change'].apply(determine_action)
final_year_averages.drop(columns=['next_year_price_yoy_pct_change'], inplace=True)

final_year_averages.columns

Index(['price_yoy_diff', 'volume_yoy_diff', 'price_yoy_pct_change',
       'volume_yoy_pct_change', '1yr_u_rate_pct_diff',
       '1yr_sentiment_pct_change', '1yr_gdp_pct_change',
       '1yr_fed_funds_pct_change', '1yr_cpi_pct_change', 'action'],
      dtype='object')

remove the non percent changes because those can get negatively effected 

In [70]:
final_year_modeling_data = final_year_averages[['price_yoy_pct_change',
       'volume_yoy_pct_change', '1yr_u_rate_pct_diff',
       '1yr_sentiment_pct_change', '1yr_gdp_pct_change',
       '1yr_fed_funds_pct_change', '1yr_cpi_pct_change', 'action']]
final_year_modeling_data = final_year_modeling_data.reset_index()
final_year_modeling_data

,year,price_yoy_pct_change,volume_yoy_pct_change,1yr_u_rate_pct_diff,1yr_sentiment_pct_change,1yr_gdp_pct_change,1yr_fed_funds_pct_change,1yr_cpi_pct_change,action
0,1951,29.029773,-15.020485,-35.340175,NaN,10.564877,NaN,NaN,buy
1,1952,19.887514,-24.103316,-7.507572,NaN,5.165332,NaN,NaN,sell
2,1953,4.972609,5.171929,-2.239049,-2.474865,6.189877,NaN,NaN,buy
3,1954,18.549472,76.504025,97.397452,-0.118953,-1.781925,NaN,NaN,buy
4,1955,55.606807,74.416946,-21.375729,16.990869,6.170199,NaN,NaN,buy
...,...,...,...,...,...,...,...,...,...
69,2020,14.889219,38.676798,121.924519,-15.046015,1.226555,-0.953299,-31.929283,buy
70,2021,42.304832,8.150914,-17.346642,-4.813490,1.572607,-0.196831,280.829952,hold
71,2022,9.473566,5.095595,-30.502669,-24.017608,3.565082,20.089552,70.349941,sell
72,2023,-1.571161,-6.833823,-0.016389,10.809665,1.717927,14.323897,NaN,buy
